In [1]:
import time
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')

from collections import OrderedDict
import functools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
# use ggplot styles for graphs
plt.style.use('ggplot')

import pickle
import arviz as az

In [2]:
import ot
import tensorflow as tf
import tensorflow_probability as tfp

# set tf logger to log level ERROR to avoid warnings
tf.get_logger().setLevel('ERROR')

tfd = tfp.distributions
tfb = tfp.bijectors
tfk = tf.keras

In [3]:
# import probabilistic models
from bayes_vi.models import Model

# import utils
from bayes_vi.utils.datasets import make_dataset_from_df

In [4]:
# mcmc imports
from bayes_vi.inference.mcmc import MCMC
from bayes_vi.inference.mcmc.transition_kernels import HamiltonianMonteCarlo, NoUTurnSampler, RandomWalkMetropolis
from bayes_vi.inference.mcmc.stepsize_adaptation_kernels import SimpleStepSizeAdaptation, DualAveragingStepSizeAdaptation

In [5]:
# vi imports 
from bayes_vi.inference.vi import VI

from bayes_vi.inference.vi.surrogate_posteriors import ADVI, NormalizingFlow
from bayes_vi.utils import to_ordered_dict
from bayes_vi.inference.vi.flow_bijectors import HamiltonianFlow, AffineFlow, make_energy_fn, make_scale_fn, make_shift_fn
from bayes_vi.utils.leapfrog_integrator import LeapfrogIntegrator

# Experiment 2 - Simple Linear Regression

## MCMC Config

In [6]:
# sampling params
NUM_CHAINS = 5
NUM_SAMPLES = 1000
NUM_BURNIN_STEPS = 10000

In [7]:
# define step size adaptation
stepsize_adaptation_kernel = DualAveragingStepSizeAdaptation(num_adaptation_steps=int(NUM_BURNIN_STEPS*0.8))

kernel = NoUTurnSampler(
    step_size=0.01, 
    max_tree_depth=5,
    stepsize_adaptation_kernel=stepsize_adaptation_kernel
)

## VI Config

In [8]:
NUM_STEPS = 10000
SAMPLE_SIZE = 50

In [9]:
optimizer = tf.optimizers.Adam
init_lr = {
    'meanfield advi': 5e-3, 
    'affine flow': 5e-3,
    'maf': 5e-3,
    'cnf': 1e-4,
    'hnf(1)': 1e-3,
    'hnf(2)': 1e-3,
    'hnf(3)': 1e-3
}

In [10]:
func_dict = {
    "mean": np.mean,
    "sd": np.std,
    "min": lambda x: np.percentile(x, 0),
    "hdi 3%": lambda x: np.percentile(x, 3),
    "mode": lambda x: np.percentile(x, 50),
    "hdi 97%": lambda x: np.percentile(x, 97),
    "max": lambda x: np.percentile(x, 100)
}

In [11]:
def get_continuous_flow_bijector(unconstrained_event_dims):
    state_fn = tfk.Sequential()
    state_fn.add(tfk.layers.Dense(128, activation=tfk.activations.tanh))
    state_fn.add(tfk.layers.Dense(128, activation=tfk.activations.tanh))
    state_fn.add(tfk.layers.Dense(unconstrained_event_dims))
    state_fn.build((None, unconstrained_event_dims+1))
    state_time_derivative_fn = lambda t, state: state_fn(tf.concat([tf.fill((state.shape[0],1), t), state], axis=-1))
    return tfb.FFJORD(state_time_derivative_fn, 
                      ode_solve_fn=tfp.math.ode.DormandPrince(first_step_size=0.1).solve, 
                      trace_augmentation_fn=tfb.ffjord.trace_jacobian_hutchinson)
    
def get_hamiltonian_flow_bijector(unconstrained_event_dims, num_flows):
    return tfb.Chain([
        HamiltonianFlow(
            event_dims=unconstrained_event_dims,
            symplectic_integrator=LeapfrogIntegrator(), 
            step_sizes=tf.Variable(0.1), 
            num_integration_steps=2,
            hidden_layers=[128, 128]
        ) for _ in range(num_flows)
    ])

def get_masked_autoregressive_flow_bijector(unconstrained_event_dims):
    return tfb.MaskedAutoregressiveFlow(
        shift_and_log_scale_fn=tfb.AutoregressiveNetwork(params=2, hidden_units=[128, 128], activation='tanh')
    )

def get_affine_flow_bijector(unconstrained_event_dims):
    return AffineFlow(unconstrained_event_dims)

def get_posterior_lift_distribution(unconstrained_event_dims):
    scale_fn = make_scale_fn(unconstrained_event_dims, hidden_layers=[128,128])
    shift_fn = make_shift_fn(unconstrained_event_dims, hidden_layers=[128,128])
    return lambda q: tfd.MultivariateNormalDiag(loc=shift_fn(q), scale_diag=scale_fn(q))

## Define Model

In [12]:
# define priors
priors = OrderedDict(
    beta_0 = tfd.Normal(loc=0., scale=10.),
    beta_1 = tfd.Normal(loc=0., scale=10.),
    scale = tfd.HalfNormal(scale=10.),
)

# define likelihood
def likelihood(beta_0, beta_1, scale, features):
    linear_response = beta_0 + beta_1*features['x']
    return tfd.Normal(loc=linear_response, scale=scale)

In [13]:
model = Model(priors=priors, likelihood=likelihood)

## Generate Datasets

In [14]:
num_datasets = 10
num_datapoints = 100

In [15]:
true_params = {}
datasets = {}

for i in range(1, num_datasets+1):
    x = tf.random.normal(shape=(num_datapoints, ), mean=0., stddev=10.0, dtype=tf.float32)
    dist = model.get_joint_distribution(features={'x': x})
    beta_0, beta_1, scale, y = dist.sample().values()
    
    true_params['dataset {}'.format(i)] = to_ordered_dict(
        model.param_names, [beta_0.numpy(), beta_1.numpy(), scale.numpy()]
    ) 
    datasets['dataset {}'.format(i)] = make_dataset_from_df(
        pd.DataFrame({'y': y, 'x': x}), target_names=['y'], feature_names=['x'], format_features_as='dict'
    ) 

## Run Inferences and Collect Results

In [17]:
results = {}

In [18]:
for i, (name, dataset) in enumerate(datasets.items()):
    print('Starting with {}\n'.format(name))
    features, targets = list(dataset.batch(dataset.cardinality()).take(1))[0]
    
    results[name]={
        'data': {
            'features': {k: v.numpy() for k,v in features.items()},
            'targets': targets.numpy(),
            'true params': true_params[name]
        }}

    ################################################################################################################################
    
    # MCMC
    print('Start MCMC...')

    mcmc = MCMC(model=model, dataset=dataset, transition_kernel=kernel)
    
    print('Run NUTS:')
    start = time.time()
    mcmc_result = mcmc.fit(
        num_chains=NUM_CHAINS, 
        num_samples=NUM_SAMPLES, 
        num_burnin_steps=NUM_BURNIN_STEPS,
        progress_bar=True,
    )
    run_time = round(time.time() - start, 2)
    print('Finished in {}s\n'.format(run_time))

    post_pred_dist = model.get_posterior_predictive_distribution(
        posterior_samples=mcmc_result.samples, features=features
    )
    posterior_predictive_samples = post_pred_dist.sample(100)['y'].numpy()

    posterior_samples = to_ordered_dict(model.param_names, tf.nest.map_structure(lambda v: np.swapaxes(v.numpy(), 0, 1), mcmc_result.samples))
    map_trace = lambda v: np.swapaxes(v.numpy(), 0, 1) if tf.rank(v) >= 2 else v
    traces = {k: map_trace(v) if not tf.nest.is_nested(v) else tf.nest.map_structure(map_trace, v) for k,v in mcmc_result.trace.items()}
    summary = az.summary(posterior_samples, round_to=2, kind='diagnostics', stat_funcs=func_dict, extend=True)
    cols = summary.columns.tolist()
    summary = summary[cols[-len(func_dict):] + cols[:-len(func_dict)]]

        
    mcmc_results = {
        'posterior samples': posterior_samples,
        'traces': traces,
        'posterior predictive samples': posterior_predictive_samples,
        'acceptance ratios per chain': mcmc_result.accept_ratios.numpy(),
        'summary': summary,
        'run-time': run_time
    }
    
    results[name]['mcmc'] = mcmc_results
    
    print('Finished MCMC! \n')
    ################################################################################################################################
    
    # VI
    unconstrained_event_dims = model.flat_unconstrained_param_event_ndims    

    surrogate_posteriors = {
        'meanfield advi': ADVI(model, mean_field=True), 
        'affine flow': NormalizingFlow(model, flow_bijector=get_affine_flow_bijector(unconstrained_event_dims)),
        'maf': NormalizingFlow(model, flow_bijector=get_masked_autoregressive_flow_bijector(unconstrained_event_dims)),
        'cnf': NormalizingFlow(model, flow_bijector=get_continuous_flow_bijector(unconstrained_event_dims)),
        'hnf(1)': NormalizingFlow(
            model, flow_bijector=get_hamiltonian_flow_bijector(unconstrained_event_dims, num_flows=1), 
            extra_ndims=unconstrained_event_dims, posterior_lift_distribution=get_posterior_lift_distribution(unconstrained_event_dims)),
        'hnf(2)': NormalizingFlow(
            model, flow_bijector=get_hamiltonian_flow_bijector(unconstrained_event_dims, num_flows=2), 
            extra_ndims=unconstrained_event_dims, posterior_lift_distribution=get_posterior_lift_distribution(unconstrained_event_dims)),
        'hnf(3)': NormalizingFlow(
            model, flow_bijector=get_hamiltonian_flow_bijector(unconstrained_event_dims, num_flows=3), 
            extra_ndims=unconstrained_event_dims, posterior_lift_distribution=get_posterior_lift_distribution(unconstrained_event_dims))
    }
    
    vi_results = {}
    
    print('Start VI...')
    for surrogate_name, surrogate_posterior in surrogate_posteriors.items():
    
        vi = VI(model, dataset, surrogate_posterior)
        
        print('Run {}:'.format(surrogate_name))
        start = time.time()
        approx_posterior, losses = vi.fit(optimizer=optimizer(init_lr[surrogate_name]), num_steps=NUM_STEPS, sample_size=SAMPLE_SIZE, progress_bar=True)
        run_time = round(time.time() - start, 2)
        print('Finished in {}s\n'.format(run_time))

        print('computing error metrics...')
        start = time.time()

        post_pred_dist = model.get_posterior_predictive_distribution(
            posterior_distribution=approx_posterior, features=features
        )
        posterior_predictive_samples = post_pred_dist.sample(100)['y'].numpy()
        posterior_samples = tf.nest.map_structure(lambda v: np.expand_dims(v.numpy(),0), approx_posterior.sample(5000))
        
        summary = az.summary(posterior_samples, stat_funcs=func_dict, extend=False)
        
        # compute Wasserstein distance
        flat_mcmc_samples = tf.nest.map_structure(lambda v: np.expand_dims(v.flatten(), 1), mcmc_results['posterior samples'])
        flat_vi_samples = tf.nest.map_structure(lambda v: np.expand_dims(v.flatten(), 1), posterior_samples)
        distances = tf.nest.map_structure(ot.dist, flat_mcmc_samples, flat_vi_samples)
        W2 = tf.nest.map_structure(
            lambda x,y, dist: np.sqrt(ot.emd2(ot.unif(x.size), ot.unif(y.size), dist)), 
            flat_mcmc_samples, flat_vi_samples, distances
        )
        
        # compute absolute posterior mean difference to mcmc result
        posterior_mean_error = tf.nest.map_structure(lambda x,y: np.abs(x.mean() - y.mean()), flat_vi_samples, flat_mcmc_samples)
        posterior_sd_error = tf.nest.map_structure(lambda x,y: np.abs(x.std() - y.std()), flat_vi_samples, flat_mcmc_samples)
        comp_time = round(time.time() - start, 2)
        print('Finished computing error metrics in {}s\n\n'.format(comp_time))
        
        vi_result = {
            'losses': losses,
            'final loss': losses[-100:].mean(),
            'learning rate': init_lr[surrogate_name],
            'posterior samples': posterior_samples,
            'posterior predictive samples': posterior_predictive_samples,
            'summary': summary,
            'W2': W2,
            'mean error':  posterior_mean_error,
            'sd error': posterior_sd_error,
            'run-time': run_time,
        }
        vi_results[surrogate_name] = vi_result
    print('Finished VI! \n\n\n\n')
    results[name]['vi'] = vi_results
    

Starting with dataset 1

Start MCMC...
Run NUTS:


Finished in 203.48s

Finished MCMC! 

Start VI...
Run meanfield advi:


Finished in 14.86s

computing error metrics...


arviz - WARNING - Shape validation failed: input_shape: (1, 5000), minimum_shape: (chains=2, draws=4)


Finished computing error metrics in 18.44s


Run affine flow:


...

arviz - WARNING - Shape validation failed: input_shape: (1, 5000), minimum_shape: (chains=2, draws=4)


Finished computing error metrics in 22.64s


Run hnf(3):


Finished in 169.78s

computing error metrics...


arviz - WARNING - Shape validation failed: input_shape: (1, 5000), minimum_shape: (chains=2, draws=4)


Finished computing error metrics in 13.94s


Finished VI! 






In [19]:
with open('./linear_regression_experiment/linear_regression_exp_results.pickle', 'wb') as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)